In this notebook I will use the [Chinook Database](https://github.com/lerocha/chinook-database) to answer a bunch of simple questions. First, let's
connect to the database.

In [18]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
cred = {"host": 'localhost', 'dbname': 'yukontaf', 'user': 'glebsokolov',
        'password': ''}
from sqlalchemy import create_engine

con = create_engine(
    f'postgresql://{cred["user"]}:{cred["password"]}@{cred["host"]}/{cred["dbname"]}'
)


def select(sql):
    return pd.read_sql(sql, con)

The scheme of the database looks like this ![this](https://previews.dropbox.com/p/thumb/ABgcFGgWyXLXx2VvCYJfBa9uEX7RRcgXKk2cWScIcTe4Y0qcAFlSoVVDBguIqI9Ar8fNiIl75haODx8KnaIXnSI8WP6JL-HjObr6_8z6NkaI6m54qgzXreDW1f98yQCpdh-xor27CS6KR1D7oggpmobFxkkl4i_f-9w9UfD49t4Y7KH8qyLf2V0NCKny-WyS9AwnoJtio3j31RHDj4oqKl3iAaG4BGMcHPkuuq9-jrMFahYZGgiGFD_85v1RJzhvvwoUrgZeAHGDIIMFnTmEv5OqtCXLnlP88rPzI3oDuKehyFcF0Zygkpjf8kOuyIMUs4I5PSU7AM7QFRO1s3rptk8A0kUHqtT0RoTEm_lK6a2obPTsSCKPT2st6rgFGI6sR8U/p.png):

- Which countries have the most invoices?

In [6]:
sql = '''SELECT DISTINCT "BillingCountry",
                count("InvoiceId") OVER (PARTITION BY "BillingCountry") AS cnt
FROM "Invoice"
ORDER BY cnt DESC'''
select(sql)

,BillingCountry,cnt
0,USA,91
1,Canada,56
2,Brazil,35
3,France,35
4,Germany,28
5,United Kingdom,21
6,Czech Republic,14
7,Portugal,14
8,India,13
9,Argentina,7


- Which city has the best customers? (We need to find out which city has the highest
 sum of invoice totals)

In [7]:
sql = '''SELECT "BillingCity",
       sum("Total") AS total
FROM "Invoice"
GROUP BY "BillingCity"
ORDER BY total DESC
LIMIT 1'''
select(sql)

,BillingCity,total
0,Prague,90.24


- Who is the best customer? (Build a query that returns the person who has spent the
most money)

In [8]:
sql = '''SELECT DISTINCT "CustomerId" ,
                sum("Total") OVER (PARTITION BY "CustomerId") AS total_sum
FROM "Invoice" a
LEFT JOIN "InvoiceLine" b ON a."InvoiceId" = b."InvoiceId"
ORDER BY total_sum DESC'''
select(sql)

,CustomerId,total_sum
0,6,502.62
1,26,474.62
2,45,446.62
3,46,446.62
4,57,415.62
5,25,404.62
6,7,404.62
7,37,388.62
8,24,378.62
9,43,376.62


- Find out the email, first name, last name, and Genre of all Rock Music listeners

In [13]:
sql = '''SELECT "FirstName",
       "LastName",
       "Email",
       e."Name"
FROM "Invoice" a
         LEFT JOIN "InvoiceLine" b ON a."InvoiceId" = b."InvoiceId"
         LEFT JOIN "Customer" c ON a."CustomerId" = c."CustomerId"
         LEFT JOIN "Track" d ON b."TrackId" = d."TrackId"
         LEFT JOIN "Genre" e ON d."GenreId" = e."GenreId"'''
select(sql)

,FirstName,LastName,Email,Name
0,Aaron,Mitchell,aaronmitchell@yahoo.ca,Rock
1,Alexandre,Rocha,alero@uol.com.br,Rock
2,Astrid,Gruber,astrid.gruber@apple.at,Rock
3,Bj�rn,Hansen,bjorn.hansen@yahoo.no,Rock
4,Camille,Bernard,camille.bernard@yahoo.fr,Rock
5,Daan,Peeters,daan_peeters@apple.be,Rock
6,Diego,Guti�rrez,diego.gutierrez@yahoo.ar,Rock
7,Dan,Miller,dmiller@comcast.com,Rock
8,Dominique,Lefebvre,dominiquelefebvre@gmail.com,Rock
9,Edward,Francis,edfrancis@yachoo.ca,Rock


- Now let's find  the artists who have written the most rock music in our dataset.
So, I will write a query that returns the Artist name and total track count of the top
 10 rock bands.

In [15]:
sql = '''SELECT DISTINCT a."Name",
                count(c."Name") OVER (PARTITION BY a."Name") AS song_cnt
FROM "Artist" a
         LEFT JOIN "Album" b ON a."ArtistId" = b."ArtistId"
         LEFT JOIN "Track" c ON b."AlbumId" = c."AlbumId"
         LEFT JOIN "Genre" d ON c."GenreId" = d."GenreId"
WHERE d."Name" = 'Rock'
ORDER BY song_cnt DESC'''
select(sql)

,Name,ArtistId,song_cnt
0,Led Zeppelin,22,114
1,U2,150,112
2,Deep Purple,58,92
3,Iron Maiden,90,81
4,Pearl Jam,118,54
5,Van Halen,152,52
6,Queen,51,45
7,The Rolling Stones,142,41
8,Creedence Clearwater Revival,76,40
9,Kiss,52,35


- Find out which artist has earned the most according to the InvoiceLines
- Use this artist to find which customer spent the most on this artist

In [21]:
sql1 = '''SELECT DISTINCT e."ArtistId",
                e."Name",
                sum(b."UnitPrice"*"Quantity") OVER (PARTITION BY e."ArtistId") AS total_earings
FROM "Invoice" a
LEFT JOIN "InvoiceLine" b ON a."InvoiceId" = b."InvoiceId"
LEFT JOIN "Track" c ON b."TrackId" = c."TrackId"
LEFT JOIN "Album" d ON c."AlbumId" = d."AlbumId"
LEFT JOIN "Artist" e ON d."ArtistId" = e."ArtistId"
ORDER BY total_earings DESC'''
select(sql1)

,ArtistId,Name,total_earings
0,90,Iron Maiden,138.60
1,150,U2,105.93
2,50,Metallica,90.09
3,22,Led Zeppelin,86.13
4,149,Lost,81.59
...,...,...,...
160,200,The Posies,0.99
161,260,Adrian Leaper & Doreen de Feis,0.99
162,264,Kent Nagano and Orchestre de l'Op�ra de Lyon,0.99
163,238,Maurizio Pollini,0.99


In [22]:
sql2 = '''SELECT DISTINCT "CustomerId",
                sum(b."UnitPrice"*"Quantity") OVER (PARTITION BY "CustomerId") AS customer_spendings
FROM "Invoice" a
         LEFT JOIN "InvoiceLine" b ON a."InvoiceId" = b."InvoiceId"
         LEFT JOIN "Track" c ON b."TrackId" = c."TrackId"
         LEFT JOIN "Album" d ON c."AlbumId" = d."AlbumId"
         LEFT JOIN "Artist" e ON d."ArtistId" = e."ArtistId"
WHERE e."Name"='Iron Maiden'
ORDER BY customer_spendings DESC'''
select(sql2)

,CustomerId,customer_spendings
0,55,17.82
1,35,15.84
2,16,13.86
3,36,13.86
4,5,8.91
5,27,8.91
6,19,5.94
7,39,5.94
8,58,5.94
9,7,4.95


- Return all the track names that have a song length longer than the average song
length.

In [23]:
sql = '''SELECT tt."Name",
       tt."Milliseconds"
FROM
    (SELECT t."Name",
            "Milliseconds",
            AVG ("Milliseconds") OVER (PARTITION BY t .col) AS average
     FROM
         (SELECT "Name",
                 1 AS col,
                 "Milliseconds"
          FROM "Track")t)tt
WHERE tt."Milliseconds">tt.average
ORDER BY "Milliseconds" DESC'''
select(sql)

,Name,Milliseconds
0,Occupation / Precipice,5286953
1,Through a Looking Glass,5088838
2,"Greetings from Earth, Pt. 1",2960293
3,The Man With Nine Lives,2956998
4,"Battlestar Galactica, Pt. 2",2956081
...,...,...
489,22 Acacia Avenue,395572
490,The Unforgiven II,395520
491,The Shortest Straw,395389
492,"Concerto for Clarinet in A Major, K. 622: II. ...",394482
